In [ ]:
# Author: Your name
# Email: Your email
# Date: 

# **Assignment**
1. File -> "save a copy in Drive" and open the copy file.
2. Write down a question and document.
3. Run the code and return the token<->token_id pairs.
4. Run the model.
5. One line code change: Instead of returning the span answer, just return the answer starting position id and ending position id in the paragraph. 
6. Return the answer.
7. Save this .ipynb and submit.

In [ ]:
!pip install transformers

In [ ]:
## load an example. The input requires question and paragraph text.
## load an example. The input requires question and paragraph text.
question = ""  ### choose a question by yourself.
answer_text = "" ### choose a document by yourself (less than 512 tokens).

In [ ]:
# !pip install ipykernel
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
##########################################################################################################################
# In this task, we can choose a famous dataset, SQuAD, as the Reading Comprehension benchmark.
# The data link is shown in here: https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json .
##########################################################################################################################  

In [ ]:
## load the pre-trained QA model and tokenizer from Huggingface
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [ ]:
### transfer the token to token id
input_ids = tokenizer.encode(question, answer_text)
print('The input has a total of {:} tokens.'.format(len(input_ids)))

In [ ]:
## if you want, you can change back to token for doule checking.
tokens = tokenizer.convert_ids_to_tokens(input_ids)
print(tokens)

In [ ]:
### or more dircetly, print token<->id format
for token, id in zip(tokens, input_ids):
    if id == tokenizer.sep_token_id:
        print('')
    print('{:<12} {:>6,}'.format(token, id))
    if id == tokenizer.sep_token_id:
        print('')

In [ ]:
## create segment id (important) for question (all 0) and paragraph text (all 1)
sep_index = input_ids.index(tokenizer.sep_token_id)

num_seg_a = sep_index + 1 # The number of segment A tokens includes the [SEP] token istelf.

num_seg_b = len(input_ids) - num_seg_a # The remainder are segment B.

segment_ids = [0]*num_seg_a + [1]*num_seg_b # Construct the list of 0s and 1s.

assert len(segment_ids) == len(input_ids)

In [ ]:
### most exciting code: run the BERT model!
# Run our example through the model.
outputs = model(torch.tensor([input_ids]),
                             token_type_ids=torch.tensor([segment_ids]),
                             return_dict=True) 
start_scores = outputs.start_logits
end_scores = outputs.end_logits

In [ ]:
## let's see the output returning result.
print(outputs)

In [ ]:
### find the answer span through the logit scores and the argmax operation
answer_start = torch.argmax(start_scores)
answer_end = torch.argmax(end_scores)

### Hint: print the answer starting position id index and answer ending position id index in the paragraph here.
# print('answer starting position id: ', ? , ' answer ending position id: ', ?)

# Combine the tokens in the answer and print it out.
answer = ' '.join(tokens[answer_start:answer_end+1])

print('Answer: "' + answer + '"')